## Data Processing

In [ ]:
import pandas as pd

In [ ]:
bucket = ""

In [ ]:
#file = "cleaned_historical_trips_2015_2022.csv"

#s3_data_location = f"s3://{bucket}/{file}*"
#trips = pd.read_csv(s3_data_location, parse_dates = True)

In [ ]:
#trips = trips[(trips["starttime"] > "2017-09-01") & (trips["stoptime"] < "2022-08-31")]

In [ ]:
#trips_start = trips[["starttime", "start station id", "start station name"]]
#trips_stop = trips[["stoptime", "end station id", "end station name"]]

In [ ]:
#trips_start.to_csv("model_trips_start_station_2017_2022.csv")
#trips_stop.to_csv("model_trips_stop_station_2017_2022.csv")

### Trip Start Station

In [ ]:
start_file = "model_trips_start_station_2017_2022.csv"

s3_start_location = f"s3://{bucket}/{start_file}*"
trips_start = pd.read_csv(s3_start_location, parse_dates = True)

In [ ]:
trips_start.shape

Check start station id matches up with start station name. In this case, some stations have changed names due to location changes or due to a lack of data standardization. `trips_start_lookup` is a lookup table to match between the different station ids and station names.

In [ ]:
#trips_start["start station id"].nunique()

In [ ]:
#trips_start["start station name"].nunique()

In [ ]:
#trips_start.drop_duplicates(subset = ["start station id", "start station name"]).to_csv("unique_start.csv")

In [ ]:
#trips_start_lookup = trips_start.drop(["Unnamed: 0", "starttime"], axis = 1).drop_duplicates()

In [ ]:
#trips_start_lookup.to_csv("trip_start_station_id_lookup.csv", index = False)

Will use start station id and not start station name. From manually looking at the data, station name has more variation and very similar station names have the same station id.

In [ ]:
trips_start = trips_start.drop(["Unnamed: 0", "start station name"], axis = 1)

In [ ]:
trips_start["starttime"] = pd.to_datetime(trips_start["starttime"])

In [ ]:
trips_start

In [ ]:
trips_start_group = trips_start.groupby(["starttime", "start station id"], as_index = False).size()

In [ ]:
sum(trips_start_group["size"])

In [ ]:
trips_start_group.set_index("starttime", inplace = True)

In [ ]:
trip_start_15min = trips_start_group.groupby([pd.Grouper(freq = "15min"), "start station id"])

In [ ]:
trip_start_15min = trip_start_15min["size"].sum().unstack("start station id")

In [ ]:
trip_start_15min

In [ ]:
sum(trip_start_15min.sum())

Find when the first trip took place for each station.

In [ ]:
trips_start.join(trips_start.groupby("start station id")["starttime"].agg(['min', 'max']), on = "start station id")[["start station id", "min"]].drop_duplicates()

### Trip End Station

In [ ]:
end_file = "model_trips_stop_station_2017_2022.csv"

s3_end_location = f"s3://{bucket}/{end_file}*"
trips_stop = pd.read_csv(s3_end_location, parse_dates = True)